# Train a deep CNN on XPS data on Google Colab

In this notebook, we will train a deep convolutional network on iron XPS spectra using the local CPU/GPU.

## Setup

### Install packages and import modules

In [ ]:
# Pip install packages
!pip install tensorflow==2.1.0 #only for now since the progress bar looks bad
!pip install python-docx
import os

# Import standard modules and magic commands
import datetime
import numpy as np
import pytz
from IPython.core.interactiveshell import InteractiveShell
%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### Mount google drive, change working directory

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

# Change working path
os.chdir(os.path.join(os.getcwd().partition('content')[0],
                      'content/drive/My Drive/app'))

### Load custom modules

In [ ]:
# Import custom classes
from xpsdeeplearning.network.classifier import Classifier
from xpsdeeplearning.network.utils import TrainingGraphs, Report

## Classification

### Setting up the parameters & folder structure

In [ ]:
np.random.seed(502)
time =  datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
model_type = 'CNN_simple'
model_name = 'Fe_single_4_classes_CNN_simple'

input_filepath = '/content/drive/My Drive/app/datasets/20200605_iron_single_small.h5'
label_values = ['Fe metal', 'FeO', 'Fe3O4', 'Fe2O3']

clf = Classifier(time = time,
                 model_type = model_type,
                 model_name = model_name,
                 labels = label_values)

### Load and inspect the data

In [ ]:
train_test_split = 0.2
train_val_split = 0.2
no_of_examples = 1000

X_train, X_val, X_test, y_train, y_val, y_test = \
        clf.load_data_preprocess(input_filepath = input_filepath,
                                 no_of_examples = no_of_examples,
                                 train_test_split = train_test_split,
                                 train_val_split = train_val_split)
        
# Check how the examples are distributed across the classes.
class_distribution = clf.check_class_distribution()
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 9, dataset = 'train')  

### Build the model

In [ ]:
clf.build_model()
clf.summary()
clf.save_and_print_model_image()

### Training

In [ ]:
epochs = 20
batch_size = 32

hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs, 
                 batch_size = batch_size,
                 verbose = 1)

### Graphs

In [ ]:
graph = TrainingGraphs(clf.history,
                        clf.model_name,
                        clf.time) 

### Evaluation on test data

In [ ]:
score = clf.evaluate()
test_loss, test_accuracy = score[0], score[1]
print('Test loss: ' + str(np.round(test_loss, decimals=3)))
print('Test accuracy: ' + str(np.round(test_accuracy, decimals=3)))

###  Prediction on test data

In [ ]:
pred_train, pred_test = clf.predict()
pred_train_classes, pred_test_classes = clf.predict_classes()

### Show some predictions

#### 5 random training samples

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  

#### 5 random test samples

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

### Show wrong predictions

In [ ]:
clf.plot_wrong_classification()

### Saving model and data

In [ ]:
clf.save_model()
clf.shelve_results(full = False)  

### Report

In [ ]:
dir_name = clf.time + '_' + clf.model_name
rep = Report(dir_name)  
rep.write()   

### Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"), include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time

save_notebook()
time.sleep(3)
current_file = 'train_colab.ipynb'
output_file = os.path.join(clf.log_dir,'train_colab_out.html')
output_HTML(current_file, output_file)
